In [4]:
import sys  
sys.path.append("../../")
from LIB.utils import * 

In [5]:

v="v6"

train_df=pd.read_csv("../features/train_{}.csv".format(v))
test_df=pd.read_csv("../features/test_{}.csv".format(v))

In [6]:
# train_df.Sales_first_7_days=np.log1p(train_df.Sales_first_7_days)
# test_df.Sales_first_7_days=np.log1p(test_df.Sales_first_7_days)

In [7]:
train_df.columns

Index(['ID', 'month', 'sku_hash', 'product_type', 'product_gender',
       'macro_function', 'function', 'sub_function', 'model',
       'aesthetic_sub_line',
       ...
       'Sales_first_7_days', 'first_month_sales', 'ratio',
       'count_sales_per_month', 'price_ratio', 'views_count',
       'TotalBuzzPost_7_day_before', 'TotalBuzz_7_day_before',
       'NetSentiment_7_day_before', 'PositiveSentiment_7_day_before'],
      dtype='object', length=117)

In [10]:
keys= ["ID"]
Target_name="target_log"
not_used_cols=["ID",'sku_hash', 'product_type', 'product_gender',
       'macro_function', 'function', 'sub_function', 'model',
       'aesthetic_sub_line', 'macro_material', 'color', 'fr_FR_price',
       'en_US_description', 'target', 'target_log',
                'ratio_function_month1_2',
  'ratio_function_month2_3',
  'ratio_function_month1_3',"ratio","count_sales_per_month",
              
              "price_ratio","views_count",  'TotalBuzzPost_7_day_before',
  'TotalBuzz_7_day_before',
  'NetSentiment_7_day_before',
  'PositiveSentiment_7_day_before']
    

feautres_name=[ f for f in test_df.columns if f not in not_used_cols]
from sklearn.metrics import mean_squared_error
def eval_metrics(y_ture,y_hat):
    return np.sqrt(mean_squared_error(y_ture,y_hat))


In [11]:
len(feautres_name),feautres_name

(94,
 ['month',
  'target_log_by_product_type_mean',
  'target_log_by_product_type_min',
  'target_log_by_product_type_std',
  'target_log_by_product_gender_mean',
  'target_log_by_product_gender_min',
  'target_log_by_product_gender_std',
  'target_log_by_macro_function_mean',
  'target_log_by_macro_function_min',
  'target_log_by_macro_function_std',
  'target_log_by_function_mean',
  'target_log_by_function_min',
  'target_log_by_function_std',
  'target_log_by_sub_function_mean',
  'target_log_by_sub_function_min',
  'target_log_by_sub_function_std',
  'target_log_by_aesthetic_sub_line_mean',
  'target_log_by_aesthetic_sub_line_min',
  'target_log_by_aesthetic_sub_line_std',
  'target_log_by_macro_material_mean',
  'target_log_by_macro_material_min',
  'target_log_by_macro_material_std',
  'type_tf_idf_pca_0',
  'type_tf_idf_pca_1',
  'type_tf_idf_pca_2',
  'type_tf_idf_pca_3',
  'type_tf_idf_pca_4',
  'type_tf_idf_pca_5',
  'type_tf_idf_pca_6',
  'type_tf_idf_pca_7',
  'type_tf_id

In [31]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
K = 5
kf = KFold(n_splits = K, random_state = 2019, shuffle = True)
X_train=train_df[feautres_name].values
X_test=test_df[feautres_name].values
X_target=train_df["target_log"].values
valid=np.zeros_like(X_target)
xgb_pred=np.zeros(len(test_df))
for train_index, test_index in kf.split(train_df):
   
    train_X, valid_X = X_train[train_index,:], X_train[test_index,:]
    train_y, valid_y = X_target[train_index], X_target[test_index]

    
    model = RandomForestRegressor()
    model.fit(train_X,train_y)
    valid[test_index] =model.predict(valid_X)                
    xgb_pred = xgb_pred+model.predict(X_test)/(K)
  

In [32]:
from sklearn.metrics import mean_squared_error 
print( np.sqrt(mean_squared_error(valid,X_target)))

0.5198323254529061


In [33]:
sub=test_df[["ID"]]
sub["target"]=np.expm1(xgb_pred)

In [34]:
sub["target"]=sub["target"].round()

In [35]:
sub[["ID","target"]].to_csv("../sub/random.csv",index=False)